OBJETIVO: Ler oum arquivo CSV, selecionar as variáveis de interesse, converter as unidades e preencher as variáveis do arquivo netCDF de acordo com as instruções do manual da ARM. É importante lembrar que os atributos globais e das variáveis podem ser ajustados de acordo com as características específicas dos dados.

In [10]:
import numpy as np
import pandas as pd

import os

import act
import matplotlib.pyplot as plt

# First import the NetCDF4 package

# Next import plotting functions
from pylab import *

from datetime import date, datetime

import xarray as xr
import matplotlib.pyplot as plt




import copy
import glob
import json
import re
import urllib
import warnings
from pathlib import Path, PosixPath

import netCDF4 as nc
from netCDF4 import Dataset
from netCDF4 import MFDataset
from os import PathLike
import tarfile
import tempfile

import numpy as np
import xarray as xr

import act.utils as utils
from act.config import DEFAULT_DATASTREAM_NAME
from act.utils.io_utils import unpack_tar, unpack_gzip, cleanup_files, is_gunzip_file



%matplotlib inline

In [91]:
#Abrindo os dados e criando o Dataframe
df_mp3000A_1 = pd.read_table('./Data/level1_output_bt.txt',header=None,delim_whitespace=True)

In [92]:
df_mp3000A_1.columns = ['Mes', 'dia', 'Ano', 'H', 'M', 'S', 'Ch_22_234', 'Ch_22_500', 'Ch_23_034', 'Ch_23_834', 'Ch_25_000', 'Ch_26_234', 'Ch_28_000', 'Ch_30_000', 'Ch_51_248', 'Ch_51_760', 'Ch_52_280', 'Ch_52_804', 'Ch_53_336', 'Ch_53_848', 'Ch_54_400', 'Ch_54_940', 'Ch_55_500', 'Ch_56_020', 'Ch_56_660', 'Ch_57_288', 'Ch_57_964', 'Ch_58_800', 'DataQuality']


In [ ]:
#Faz uma busca na coluna Ano e altera o ano de YY para YYYY
df_mp3000A_1['Ano'][df_mp3000A_1['Ano'] < 2000] = df_mp3000A_1['Ano'] + 2000

In [94]:

#Modificanto o tipo de dado para (int)
df_mp3000A_1[['Mes', 'dia', 'Ano', 'H', 'M', 'S']] = df_mp3000A_1[['Mes', 'dia', 'Ano', 'H', 'M', 'S']].astype(int)

In [95]:
import calendar
import datetime

#Adiciona a coluna Datetime ao Dataframe com os dados do RD80 (Joss)
df_mp3000A_1['Datetime'] = df_mp3000A_1[['Ano', 'Mes', 'dia', 'H', 'M', 'S']].apply(lambda row:
                    datetime.datetime(year=row['Ano'], month=row['Mes'],day=row['dia'], hour=row['H'], minute=row["M"], second=row["S"]),axis=1)

In [96]:
df_mp3000A_1.columns

Index(['Mes', 'dia', 'Ano', 'H', 'M', 'S', 'Ch_22_234', 'Ch_22_500',
       'Ch_23_034', 'Ch_23_834', 'Ch_25_000', 'Ch_26_234', 'Ch_28_000',
       'Ch_30_000', 'Ch_51_248', 'Ch_51_760', 'Ch_52_280', 'Ch_52_804',
       'Ch_53_336', 'Ch_53_848', 'Ch_54_400', 'Ch_54_940', 'Ch_55_500',
       'Ch_56_020', 'Ch_56_660', 'Ch_57_288', 'Ch_57_964', 'Ch_58_800',
       'DataQuality', 'Datetime'],
      dtype='object')

In [97]:
df_mp3000A_1 = df_mp3000A_1.reindex(columns=['Datetime','Mes', 'dia', 'Ano', 'H', 'M', 'S', 'Ch_22_234', 'Ch_22_500',
       'Ch_23_034', 'Ch_23_834', 'Ch_25_000', 'Ch_26_234', 'Ch_28_000',
       'Ch_30_000', 'Ch_51_248', 'Ch_51_760', 'Ch_52_280', 'Ch_52_804',
       'Ch_53_336', 'Ch_53_848', 'Ch_54_400', 'Ch_54_940', 'Ch_55_500',
       'Ch_56_020', 'Ch_56_660', 'Ch_57_288', 'Ch_57_964', 'Ch_58_800',
       'DataQuality'])

In [98]:
df_mp3000A_1.columns

Index(['Datetime', 'Mes', 'dia', 'Ano', 'H', 'M', 'S', 'Ch_22_234',
       'Ch_22_500', 'Ch_23_034', 'Ch_23_834', 'Ch_25_000', 'Ch_26_234',
       'Ch_28_000', 'Ch_30_000', 'Ch_51_248', 'Ch_51_760', 'Ch_52_280',
       'Ch_52_804', 'Ch_53_336', 'Ch_53_848', 'Ch_54_400', 'Ch_54_940',
       'Ch_55_500', 'Ch_56_020', 'Ch_56_660', 'Ch_57_288', 'Ch_57_964',
       'Ch_58_800', 'DataQuality'],
      dtype='object')

In [99]:
#Salvando/Exportando os Dataframes para o formato CSV
#df_mp3000A_1.to_csv('./Data/2021-07-27_19-43-20_lv1.csv')

In [100]:

# Leitura do arquivo CSV
#data = pd.read_csv('http://ftp.lfa.if.usp.br/ftp/public/ATTO/Campina/MP3000/2021-07-27_19-43-20_lv1.csv', delimiter=',', skiprows=[0])
data = df_mp3000A_1


In [101]:
data.columns

Index(['Datetime', 'Mes', 'dia', 'Ano', 'H', 'M', 'S', 'Ch_22_234',
       'Ch_22_500', 'Ch_23_034', 'Ch_23_834', 'Ch_25_000', 'Ch_26_234',
       'Ch_28_000', 'Ch_30_000', 'Ch_51_248', 'Ch_51_760', 'Ch_52_280',
       'Ch_52_804', 'Ch_53_336', 'Ch_53_848', 'Ch_54_400', 'Ch_54_940',
       'Ch_55_500', 'Ch_56_020', 'Ch_56_660', 'Ch_57_288', 'Ch_57_964',
       'Ch_58_800', 'DataQuality'],
      dtype='object')

In [102]:

# Seleção das variáveis de interesse
time = pd.to_datetime(data['Datetime'], format='%Y%m%d %H%M%S')
#time = data['Record'].values
pressure = data['Ch_22_234'].values
temperature = data['Ch_22_500'].values
relative_humidity = data['Ch_30_000'].values
wind_speed = data['Ch_58_800'].values
wind_direction = data['DataQuality'].values


In [103]:
time

0        2021-07-27 18:13:43
1        2021-07-27 18:14:16
2        2021-07-27 18:14:49
3        2021-07-27 18:15:22
4        2021-07-27 18:15:54
                 ...        
639689   2022-06-01 09:17:44
639690   2022-06-01 09:18:17
639691   2022-06-01 09:18:51
639692   2022-06-01 09:19:24
639693   2022-06-01 09:19:57
Name: Datetime, Length: 639694, dtype: datetime64[ns]

In [104]:

# Conversão de unidades
relative_humidity = relative_humidity * 3.6 # de m/s para km/h


In [1]:
filename = 'ATTO_MWR_data_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.cdf'
# Criação do arquivo netCDF
with nc.Dataset(filename, 'w', format='NETCDF4_CLASSIC') as rootgrp:


    # Criação das dimensões
    time_dim = rootgrp.createDimension('time', None)

    # Criação das variáveis
    time_var = rootgrp.createVariable('time', 'f8', ('time',))
    pressure_var = rootgrp.createVariable('air_pressure', 'f4', ('time',), fill_value=-999.0)
    temperature_var = rootgrp.createVariable('air_temperature', 'f4', ('time',), fill_value=-999.0)
    relative_humidity_var = rootgrp.createVariable('relative_humidity', 'f4', ('time',), fill_value=-999.0)
    wind_speed_var = rootgrp.createVariable('wind_speed', 'f4', ('time',), fill_value=-999.0)
    wind_direction_var = rootgrp.createVariable('wind_direction', 'f4', ('time',), fill_value=-999.0)

    # Definição dos atributos globais
    rootgrp.Conventions = 'CF-1.6, ACDD-1.3'
    rootgrp.title = 'Dados da estação MP3000 da ATTO em Campina'
    rootgrp.summary = 'Dados horários de pressão atmosférica, temperatura do ar, umidade relativa, velocidade e direção do vento medidos pela estação MP3000 da ATTO em Campina.'
    rootgrp.institution = 'National Institute for Space Research (INPE), Universidade de São Paulo, Instituto de Física'
    rootgrp.source = 'Estação MP3000 da ATTO -Campina'
    rootgrp.references = 'https://www.arm.gov/capabilities/instruments/mwr'
    rootgrp.comment = 'Data collected during the ATTO project'

    # Definição dos atributos das variáveis
    time_var.units = 'seconds since 1970-01-01 00:00:00'
    time_var.standard_name = 'time'
    pressure_var.units = 'hPa'
    pressure_var.standard_name = 'air_pressure'
    temperature_var.units = 'degree_Celsius'
    temperature_var.standard_name = 'air_temperature'
    relative_humidity_var.units = 'percent'
    relative_humidity_var.standard_name = 'relative_humidity'
    wind_speed_var.units = 'kilometers_per_hour'
    wind_speed_var.standard_name = 'wind_speed'
    wind_direction_var.units = 'degree'
    wind_direction_var.standard_name = 'wind_direction'

    # Preenchimento das variáveis com os dados
    #time_var[:] = nc.date2num(time, units=time_var.units)
    time_var[:] = time
    pressure_var[:] = pressure
    temperature_var[:] = temperature
    relative_humidity_var[:] = relative_humidity
    wind_speed_var[:] = wind_speed
    wind_direction_var[:] = wind_direction

    # Definição dos atributos específicos das variáveis
    time_var.long_name = 'Time of measurement'
    pressure_var.long_name = 'Atmospheric pressure'
    pressure_var.valid_min = np.float32(850.0)
    pressure_var.valid_max = np.float32(1100.0)
    pressure_var.comment = 'The atmospheric pressure is the pressure exerted by the weight of the atmosphere on the Earth surface.'
    temperature_var.long_name = 'Air temperature'
    temperature_var.valid_min = np.float32(-80.0)
    temperature_var.valid_max = np.float32(60.0)
    temperature_var.comment = 'The air temperature is the temperature of the air outside the instrument housing.'
    relative_humidity_var.long_name = 'Relative humidity'
    relative_humidity_var.valid_min = np.float32(0.0)
    relative_humidity_var.valid_max = np.float32(100.0)
    relative_humidity_var.comment = 'The relative humidity is the amount of water vapor present in the air relative to the amount that would be present at saturation at the same temperature.'
    wind_speed_var.long_name = 'Wind speed'
    wind_speed_var.valid_min = np.float32(0.0)
    wind_speed_var.valid_max = np.float32(100.0)
    wind_speed_var.comment = 'The wind speed is the speed of the air relative to the instrument housing.'
    wind_direction_var.long_name = 'Wind direction'
    wind_direction_var.valid_min = np.float32(0.0)
    wind_direction_var.valid_max = np.float32(360.0)
    wind_direction_var.comment = 'The wind direction is the direction from which the wind is blowing.'

print('Arquivo netCDF criado com sucesso.')

NameError: name 'datetime' is not defined

In [ ]:
rootgrp.close()

In [ ]:
import os

import act
import matplotlib.pyplot as plt

# First import the NetCDF4 package
from netCDF4 import Dataset
from netCDF4 import MFDataset

# Next import plotting functions
from pylab import *

from datetime import date, datetime

import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Access directores files downloaded
mwr_mp3000a_ds = act.io.armfiles.read_netcdf('L:/GitHub/CAP-Dissertation-Complementary-studies/Notebooks/exemplo_arm.nc')

In [ ]:
ds = xr.open_dataset('exemplo_arm.cdf', decode_times=False)

In [ ]:
ds.variables

Frozen({'time': <xarray.IndexVariable 'time' (time: 639694)>
array([1.627410e+18, 1.627410e+18, 1.627410e+18, ..., 1.654075e+18,
       1.654075e+18, 1.654075e+18])
Attributes:
    units:          seconds since 1970-01-01 00:00:00
    standard_name:  time
    long_name:      Time of measurement, 'air_pressure': <xarray.Variable (time: 639694)>
[639694 values with dtype=float32]
Attributes:
    units:          hPa
    standard_name:  air_pressure
    long_name:      Atmospheric pressure
    valid_min:      850.0
    valid_max:      1100.0
    comment:        The atmospheric pressure is the pressure exerted by the w..., 'air_temperature': <xarray.Variable (time: 639694)>
[639694 values with dtype=float32]
Attributes:
    units:          degree_Celsius
    standard_name:  air_temperature
    long_name:      Air temperature
    valid_min:      -80.0
    valid_max:      60.0
    comment:        The air temperature is the temperature of the air outside..., 'relative_humidity': <xarray.Variab

In [106]:
ds.close()

In [8]:
import netCDF4 as nc
import numpy as np

# Criar o arquivo NetCDF
ncfile = nc.Dataset("mwrp_atto_campina.nc", mode="w")

# Definir dimensões
time = ncfile.createDimension('time', None)
height = ncfile.createDimension('height', 10)
frequency = ncfile.createDimension('frequency', 3)

# Criar as variáveis
base_time_var = ncfile.createVariable("base_time", "S1")
time_offset_var = ncfile.createVariable("time_offset", "f8", dimensions=("time",))
time_var = ncfile.createVariable("time", "f8", dimensions=("time",))
qc_time_var = ncfile.createVariable("qc_time", "i4", dimensions=("time",))

# Definir atributos das variáveis
base_time_var[:] = np.string_("2015-11-30 00:00:00 0:00")
base_time_var.long_name = "Base time in Epoch"
base_time_var.units = "seconds since 1970-1-1 0:00:00 0:00"

time_offset_var.long_name = "Time offset from base_time"
time_offset_var.units = "seconds since 2015-11-30 00:00:00 0:00"

time_var.long_name = "Time offset from midnight"
time_var.units = "seconds since 2015-11-30 00:00:00 0:00"

qc_time_var.long_name = "Quality check results on field: Time offset from midnight"
qc_time_var.units = "unitless"
qc_time_var.description = "This field contains bit packed values which should be interpreted as listed. No bits set (zero) represents good data."
qc_time_var.bit_1_description = "Delta time between current and previous samples is zero."
qc_time_var.bit_1_assessment = "Indeterminate"
qc_time_var.bit_2_description = "Delta time between current and previous samples is less than the delta_t_lower_limit field attribute."
qc_time_var.bit_2_assessment = "Indeterminate"
qc_time_var.bit_3_description = "Delta time between current and previous samples is greater than the delta_t_upper_limit field attribute."
qc_time_var.bit_3_assessment = "Indeterminate"
qc_time_var.delta_t_lower_limit = 15.
qc_time_var.delta_t_upper_limit = 360.
qc_time_var.prior_sample_flag = 1
qc_time_var.comment = "If the 'prior_sample_flag' is set the first sample time from a new raw file will be compared against the time just previous to it in the stored data. If it is not set the qc_time value for the first sample will be set to 0."

#frequency.long_name = "Frequency"
#frequency.units = "gHz"
#height.long_name = "Height"
#height.units = "meters above ground level"
azimuth = ncfile.createVariable('azimuth', np.float32)
azimuth.long_name = "Azimuth angle"
azimuth.units = "degrees"
azimuth.missing_value = -9999.
elevation = ncfile.createVariable('elevation', np.float32, ('time',))
elevation.long_name = "Elevation angle"
elevation.units = "degrees"
elevation.missing_value = -9999.
surfacePressure = ncfile.createVariable('surfacePressure', np.float32, ('time',))
surfacePressure.long_name = "Surface pressure at instrument"
surfacePressure.units = "hPa"
surfacePressure.missing_value = -9999.
surfacePressure.valid_min = -180. #""
surfacePressure.valid_max = -180. #""
qc_surfacePressure = ncfile.createVariable('qc_surfacePressure', np.int32, ('time',))
qc_surfacePressure.long_name = "Quality check results on field: Surface pressure at instrument"
qc_surfacePressure.units = "unitless"
qc_surfacePressure.description = "See global attributes for individual bit descriptions."

surfaceTemperature = ncfile.createVariable('surfaceTemperature', np.float32, ('time',))
surfaceTemperature.long_name = "Surface absolute temperature at instrument"
surfaceTemperature.units = "K"
surfaceTemperature.missing_value = -9999.
surfaceTemperature.valid_min = -180.
surfaceTemperature.valid_max = -180.

qc_surfaceTemperature = ncfile.createVariable('qc_surfaceTemperature', np.int32, ('time',))
qc_surfaceTemperature.long_name = "Quality check results on field: Surface absolute temperature at instrument"
qc_surfaceTemperature.units = "unitless"
qc_surfaceTemperature.description = "See global attributes for individual bit descriptions."

surfaceRelativeHumidity = ncfile.createVariable('surfaceRelativeHumidity', np.float32, ('time',))
surfaceRelativeHumidity.long_name = "Surface relative humidity at instrument"
surfaceRelativeHumidity.units = "%"
surfaceRelativeHumidity.missing_value = -9999.
surfaceRelativeHumidity.valid_min = -180.
surfaceRelativeHumidity.valid_max = -180.

qc_surfaceRelativeHumidity = ncfile.createVariable('qc_surfaceRelativeHumidity', np.int32, ('time',))
qc_surfaceRelativeHumidity.long_name = "Quality check results on field: Surface relative humidity at instrument"
qc_surfaceRelativeHumidity.units = "unitless"
qc_surfaceRelativeHumidity.description = "See global attributes for individual bit descriptions."

surfaceWaterVaporDensity = ncfile.createVariable('surfaceWaterVaporDensity', np.float32, ('time',))
surfaceWaterVaporDensity.long_name = "Surface water vapor density at instrument"
surfaceWaterVaporDensity.units = "g/m^3"
surfaceWaterVaporDensity.missing_value = -9999.
surfaceWaterVaporDensity.valid_min = -180.
surfaceWaterVaporDensity.valid_max = -180.



# Definição dos atributos globais
ncfile.Conventions = 'CF-1.6, ACDD-1.3'
ncfile.title = 'Dados da estação MP3000 da ATTO em Campina'
ncfile.summary = 'Dados horários de pressão atmosférica, temperatura do ar, umidade relativa, velocidade e direção do vento medidos pela estação MP3000 da ATTO em Campina.'
ncfile.institution = 'National Institute for Space Research (INPE), Universidade de São Paulo, Instituto de Física'
ncfile.source = 'Estação MP3000 da ATTO -Campina'
ncfile.references = 'https://www.arm.gov/capabilities/instruments/mwr'
ncfile.comment = 'Data collected during the ATTO project'



In [7]:
# Fechar o arquivo
ncfile.close()

In [ ]:
atto_campina_mwrp = xr.open_dataset('mwrp_atto_campina.nc', decode_times=False)

In [14]:
ncfile.variables

{'base_time': <class 'netCDF4._netCDF4.Variable'>
 |S1 base_time()
     long_name: Base time in Epoch
     units: seconds since 1970-1-1 0:00:00 0:00
 unlimited dimensions: 
 current shape = ()
 filling on, default _FillValue of   used,
 'time_offset': <class 'netCDF4._netCDF4.Variable'>
 float64 time_offset(time)
     long_name: Time offset from base_time
     units: seconds since 2015-11-30 00:00:00 0:00
 unlimited dimensions: time
 current shape = (0,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'time': <class 'netCDF4._netCDF4.Variable'>
 float64 time(time)
     long_name: Time offset from midnight
     units: seconds since 2015-11-30 00:00:00 0:00
 unlimited dimensions: time
 current shape = (0,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'qc_time': <class 'netCDF4._netCDF4.Variable'>
 int32 qc_time(time)
     long_name: Quality check results on field: Time offset from midnight
     units: unitless
     description: This field contains bit 

In [10]:
ds.close()